# Python and Database

## Python DBAPI

This API has been defined to encourage similarity between the Python modules that are used to access databases.
Accessing and working with databases that provide Python modules that adhere to this specification is the same.

```
create database connection > prepare cursor > execute statement with cursor > commit with connection > close connection
```

Documentation:
- https://www.python.org/dev/peps/pep-0249/

## Database Abstraction - sqlalchemy

sqlalchemy is an abstraction layer to the underlying database. It is dealing with the database specifics and translates database statements into the respective SQL dialect.

Documentation:
- https://www.sqlalchemy.org

Tutorials:
- https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91

## Pandas to/from Database

Documentation:
- Read data: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html
- Write data: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html?highlight=to_sql#pandas.DataFrame.to_sql

Tutorials:
- https://www.dataquest.io/blog/sql-insert-tutorial/
- https://medium.com/jbennetcodes/how-to-use-pandas-to-access-databases-e4e74e6a329e

In [3]:
import pandas as pd
import sqlite3

In [4]:
connection = sqlite3.connect('TestDB1.db')

In [5]:
cars = {'name': ['Lilli','Mike','Judy','Tom'],
        'points': [2000,5000,7000,3000]
        }

df = pd.DataFrame(cars, columns= ['name', 'points'])
df

,name,points
0,Lilli,2000
1,Mike,5000
2,Judy,7000
3,Tom,3000


In [9]:
cursor = connection.cursor()
# cursor.execute('DROP TABLE scores')
# cursor.execute('CREATE TABLE scores (name text, points number)')
# sql_insert_statement = "INSERT INTO scores(name, points) VALUES(?, ?)"
# # cursor.execute(sql_insert_statement, ('Lilli', 2000))
# for _, row in df.iterrows():
#     cursor.execute(sql_insert_statement, tuple(row))
# connection.commit()

In [15]:
df.to_sql('scores', connection, if_exists='replace', index = False)

In [16]:
cursor.execute("SELECT * FROM scores").fetchall()

[('Lilli', 2000), ('Mike', 5000), ('Judy', 7000), ('Tom', 3000)]

In [17]:
pd.read_sql("SELECT * FROM scores", con=connection)

,name,points
0,Lilli,2000
1,Mike,5000
2,Judy,7000
3,Tom,3000


In [47]:
connection.close()

## ORM - Object Relational Mapping

Tutorials:
- https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/
- https://www.pythoncentral.io/sqlalchemy-orm-examples/

In [18]:
import sqlalchemy as db
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd
 
 
Base = declarative_base()

class Department(Base):
    __tablename__ = 'department'
    id = Column(Integer, primary_key=True)
    name = Column(String)
 
 
class Employee(Base):
    __tablename__ = 'employee'
    id = Column(Integer, primary_key=True)
    firstname = Column(String)
    # Use default=func.now() to set the default hiring time
    # of an Employee to be the current time when an
    # Employee record was created
    hired_on = Column(DateTime, default=func.now())
    
    department_id = Column(Integer, ForeignKey('department.id'))
    # Use cascade='delete,all' to propagate the deletion of a Department onto its Employees
    department = relationship(
        Department,
        backref=backref('employees',
                         uselist=True,
                         cascade='delete,all'))

In [19]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///orm_in_detail.sqlite')
 
from sqlalchemy.orm import sessionmaker

# create a configured "Session" class
Session = sessionmaker()
Session.configure(bind=engine)

# Create all tables
Base.metadata.create_all(engine)

In [20]:
session = Session()

In [23]:
for department_name in ["IT", "Research", "Sales"]:
    department = Department(name=department_name)
    session.add(department)
    for firstname in ["Lucy", "Bob"]:
        session.add(Employee(firstname=f"{firstname}_{department_name}", department=department))
session.commit()

In [24]:
session.query(Department).count()

3

In [25]:
session.query(Employee).count()

6

In [26]:
pd.read_sql("SELECT * FROM department", con=engine)

,id,name
0,1,IT
1,2,Research
2,3,Sales


In [27]:
pd.read_sql(db.select([Employee]), con=engine)

,id,firstname,hired_on,department_id
0,1,Lucy_IT,2020-04-29 15:42:01,1
1,2,Bob_IT,2020-04-29 15:42:01,1
2,3,Lucy_Research,2020-04-29 15:42:01,2
3,4,Bob_Research,2020-04-29 15:42:01,2
4,5,Lucy_Sales,2020-04-29 15:42:01,3
5,6,Bob_Sales,2020-04-29 15:42:01,3


In [28]:
data = pd.read_sql(db.select([db.join(Employee, Department)]), con=engine)
data

,id,firstname,hired_on,department_id,id,name
0,1,Lucy_IT,2020-04-29 15:42:01,1,1,IT
1,2,Bob_IT,2020-04-29 15:42:01,1,1,IT
2,3,Lucy_Research,2020-04-29 15:42:01,2,2,Research
3,4,Bob_Research,2020-04-29 15:42:01,2,2,Research
4,5,Lucy_Sales,2020-04-29 15:42:01,3,3,Sales
5,6,Bob_Sales,2020-04-29 15:42:01,3,3,Sales


In [29]:
metadata = db.MetaData()

emp = db.Table('emp', metadata,
              db.Column('id', db.Integer()),
              db.Column('firstname', db.String(255), nullable=False),
              db.Column('department_id', db.Integer()),
              )
dep = db.Table('dep', metadata,
              db.Column('department_id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              )

metadata.create_all(engine)

In [30]:
data.columns = ['id', 'firstname', 'hired_on', 'department_id', 'd_id', 'name']
data

,id,firstname,hired_on,department_id,d_id,name
0,1,Lucy_IT,2020-04-29 15:42:01,1,1,IT
1,2,Bob_IT,2020-04-29 15:42:01,1,1,IT
2,3,Lucy_Research,2020-04-29 15:42:01,2,2,Research
3,4,Bob_Research,2020-04-29 15:42:01,2,2,Research
4,5,Lucy_Sales,2020-04-29 15:42:01,3,3,Sales
5,6,Bob_Sales,2020-04-29 15:42:01,3,3,Sales


In [31]:
# data write for emp table
for _, row in data[["id","firstname", "department_id"]].iterrows():
    print(tuple(row))
    query = db.insert(emp).values(tuple(row)) 
    session.execute(query)
session.commit()

(1, 'Lucy_IT', 1)
(2, 'Bob_IT', 1)
(3, 'Lucy_Research', 2)
(4, 'Bob_Research', 2)
(5, 'Lucy_Sales', 3)
(6, 'Bob_Sales', 3)


In [34]:
# data write to department table
for _, row in data[["department_id","name"]].iterrows():
    print(tuple(row))
    query = db.insert(dep).values(tuple(row)) 
    session.execute(query)
session.commit()

(1, 'IT')
(1, 'IT')
(2, 'Research')
(2, 'Research')
(3, 'Sales')
(3, 'Sales')


In [32]:
pd.read_sql(db.select([emp]), con=engine)

,id,firstname,department_id
0,1,Lucy_IT,1
1,2,Bob_IT,1
2,3,Lucy_Research,2
3,4,Bob_Research,2
4,5,Lucy_Sales,3
5,6,Bob_Sales,3


In [35]:
pd.read_sql(db.select([dep]), con=engine)

,department_id,name
0,1,IT
1,1,IT
2,2,Research
3,2,Research
4,3,Sales
5,3,Sales


In [36]:
pd.read_sql(db.select([db.join(emp, dep, emp.columns.department_id == dep.columns.department_id)]), con=engine)

,id,firstname,department_id,department_id,name
0,1,Lucy_IT,1,1,IT
1,1,Lucy_IT,1,1,IT
2,2,Bob_IT,1,1,IT
3,2,Bob_IT,1,1,IT
4,3,Lucy_Research,2,2,Research
5,3,Lucy_Research,2,2,Research
6,4,Bob_Research,2,2,Research
7,4,Bob_Research,2,2,Research
8,5,Lucy_Sales,3,3,Sales
9,5,Lucy_Sales,3,3,Sales


In [13]:
session.close()

## NoSQL Databases

- MongoDB: https://www.w3schools.com/python/python_mongodb_getstarted.asp